<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Paso 1: Instalar Gradio si no lo tenés
!pip install gradio --quiet

# ✅ Paso 2: Subir ZIP desde Colab
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import io
import gradio as gr

print("📁 Subí el archivo ZIP del PIB mundial desde el Banco Mundial:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# ✅ Paso 3: Leer y preparar los datos
with zipfile.ZipFile(io.BytesIO(uploaded[filename]), 'r') as zip_ref:
    csv_file = [f for f in zip_ref.namelist() if f.startswith("API_") and "Metadata" not in f][0]
    with zip_ref.open(csv_file) as f:
        df = pd.read_csv(f, skiprows=4)

# ✅ Paso 4: Filtrar columnas necesarias
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())

# ✅ Paso 5: Función para mostrar datos
def visualizar_pib(pais):
    datos = df[df['Country Name'] == pais].drop(columns='Country Name').T
    datos.columns = ['PIB']
    datos.index.name = 'Año'
    datos = datos.dropna()
    datos.index = datos.index.astype(int)
    datos['Crecimiento %'] = datos['PIB'].pct_change() * 100
    tabla = datos.round(2).reset_index()

    # Gráfico
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(datos.index, datos['PIB'] / 1e9, marker='o', color='blue')
    ax.set_title(f'PIB de {pais} (2000–2022)')
    ax.set_xlabel('Año')
    ax.set_ylabel('PIB (miles de millones USD)')
    ax.grid(True)
    plt.tight_layout()

    return fig, tabla

# ✅ Paso 6: Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Visualizador de PIB Mundial (2000–2022)")
    dropdown = gr.Dropdown(choices=paises, label="Seleccioná un país", value=paises[0])
    boton = gr.Button("Mostrar análisis")
    grafico = gr.Plot()
    tabla = gr.Dataframe()

    boton.click(fn=visualizar_pib, inputs=dropdown, outputs=[grafico, tabla])

# ✅ Paso 7: Lanzar Gradio (modo especial para Colab)
demo.launch(share=True)  # share=True para que funcione desde Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.8 MB/s eta 0:00:00
📁 Subí el archivo ZIP del PIB mundial desde el Banco Mundial:


Saving API_NY.GDP.MKTP.CD_DS2_es_csv_v2_1992.zip to API_NY.GDP.MKTP.CD_DS2_es_csv_v2_1992.zip
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84f325eda9f449f590.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
